tackling dummies need not apply

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

import math

In [2]:
games = pd.read_csv('./data/games.csv')
players = pd.read_csv('./data/players.csv')
plays = pd.read_csv('./data/plays.csv')
tackles = pd.read_csv('./data/tackles.csv')
week1 = pd.read_csv('./data/tracking_week_1.csv')
week2 = pd.read_csv('./data/tracking_week_2.csv')
week3 = pd.read_csv('./data/tracking_week_3.csv')
week4 = pd.read_csv('./data/tracking_week_4.csv')
week5 = pd.read_csv('./data/tracking_week_5.csv')
week6 = pd.read_csv('./data/tracking_week_6.csv')
week7 = pd.read_csv('./data/tracking_week_7.csv')
week8 = pd.read_csv('./data/tracking_week_8.csv')
week9 = pd.read_csv('./data/tracking_week_9.csv')

In [3]:
'''week1_events = week1[week1['event'].notna()]
week2_events = week2[week2['event'].notna()]
week3_events = week3[week3['event'].notna()]
week4_events = week4[week4['event'].notna()]
week5_events = week5[week5['event'].notna()]
week6_events = week6[week6['event'].notna()]
week7_events = week7[week7['event'].notna()]
week8_events = week8[week8['event'].notna()]
week9_events = week9[week9['event'].notna()]'''

"week1_events = week1[week1['event'].notna()]\nweek2_events = week2[week2['event'].notna()]\nweek3_events = week3[week3['event'].notna()]\nweek4_events = week4[week4['event'].notna()]\nweek5_events = week5[week5['event'].notna()]\nweek6_events = week6[week6['event'].notna()]\nweek7_events = week7[week7['event'].notna()]\nweek8_events = week8[week8['event'].notna()]\nweek9_events = week9[week9['event'].notna()]"

In [4]:
#week_events = [week1_events, week2_events, week3_events, week4_events, week5_events, week6_events, 
#               week7_events, week8_events, week9_events]

In [5]:
#all_events = pd.concat(week_events)

In [6]:
games

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23
...,...,...,...,...,...,...,...,...,...
131,2022110607,2022,9,11/06/2022,13:00:00,WAS,MIN,17,20
132,2022110608,2022,9,11/06/2022,16:05:00,ARI,SEA,21,31
133,2022110609,2022,9,11/06/2022,16:25:00,TB,LA,16,13
134,2022110610,2022,9,11/06/2022,20:20:00,KC,TEN,20,17


In [7]:
games_weeks = games[['gameId', 'week']]

In [8]:
games_weeks

,gameId,week
0,2022090800,1
1,2022091100,1
2,2022091101,1
3,2022091102,1
4,2022091103,1
...,...,...
131,2022110607,9
132,2022110608,9
133,2022110609,9
134,2022110610,9


In [9]:
players

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan
...,...,...,...,...,...,...,...
1678,55200,6-6,266,NaN,Indiana,DT,Ryder Anderson
1679,55212,6-0,230,NaN,Iowa State,ILB,Jake Hummel
1680,55239,6-2,300,NaN,Pennsylvania,DT,Prince Emili
1681,55240,6-1,185,NaN,Buffalo,CB,Ja'Marcus Ingram


In [10]:
players['totalInches'] = (players.height.str.split("-").str[0].astype(int) * 12) + (
    players.height.str.split("-").str[1].astype(int)
)
players['mass'] = (players['totalInches']/12 * players['weight']).round(2)

In [11]:
plays

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,43293,Ezekiel Elliott,(13:31) E.Elliott right tackle to WAS 38 for 1...,1,1,10,DAL,WAS,...,0.638600,0.361400,-0.011542,0.011542,3.642571,-0.504018,NaN,NaN,NaN,NaN
12482,2022091200,3467,46189,Will Dissly,(6:08) G.Smith pass short right to W.Dissly to...,4,1,10,SEA,DEN,...,0.615241,0.384759,-0.025458,0.025458,1.434580,-0.444642,NaN,NaN,NaN,NaN
12483,2022101605,3371,44860,Joe Mixon,(9:35) (Shotgun) J.Mixon left end to CIN 47 fo...,4,1,10,CIN,NO,...,0.667054,0.332946,-0.005164,0.005164,2.115356,0.203819,NaN,NaN,NaN,NaN
12484,2022100207,2777,52449,Jonathan Taylor,(2:02) (Shotgun) J.Taylor up the middle to TEN...,3,1,10,IND,TEN,...,0.410611,0.589389,-0.046648,0.046648,3.946232,-0.976039,NaN,NaN,NaN,NaN


In [12]:
plays['gamePlayId'] = plays[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
plays['gamePlayId'] = plays['gamePlayId'].astype('int64')

In [13]:
plays = pd.merge(plays, games_weeks, on='gameId', how='left')

In [14]:
plays

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,gamePlayId,week
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN,20221009083537,5
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN,20220911033126,1
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN,20220911111148,1
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN,20221002122007,4
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN,20220919001372,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,43293,Ezekiel Elliott,(13:31) E.Elliott right tackle to WAS 38 for 1...,1,1,10,DAL,WAS,...,-0.011542,0.011542,3.642571,-0.504018,NaN,NaN,NaN,NaN,2022100204123,4
12482,2022091200,3467,46189,Will Dissly,(6:08) G.Smith pass short right to W.Dissly to...,4,1,10,SEA,DEN,...,-0.025458,0.025458,1.434580,-0.444642,NaN,NaN,NaN,NaN,20220912003467,1
12483,2022101605,3371,44860,Joe Mixon,(9:35) (Shotgun) J.Mixon left end to CIN 47 fo...,4,1,10,CIN,NO,...,-0.005164,0.005164,2.115356,0.203819,NaN,NaN,NaN,NaN,20221016053371,6
12484,2022100207,2777,52449,Jonathan Taylor,(2:02) (Shotgun) J.Taylor up the middle to TEN...,3,1,10,IND,TEN,...,-0.046648,0.046648,3.946232,-0.976039,NaN,NaN,NaN,NaN,20221002072777,4


In [15]:
tackles

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17423,2022091113,3642,43478,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1


In [16]:
tackles['gamePlayId'] = tackles[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
tackles['gamePlayId'] = tackles['gamePlayId'].astype('int64')
tackles.rename(columns={'tackle':'tackler'}, inplace=True)

In [17]:
tackles = pd.merge(tackles, games_weeks, on=['gameId'], how='left')

In [18]:
tackles

,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,gamePlayId,week
0,2022090800,101,42816,1,0,0,0,2022090800101,1
1,2022090800,393,46232,1,0,0,0,2022090800393,1
2,2022090800,486,40166,1,0,0,0,2022090800486,1
3,2022090800,646,47939,1,0,0,0,2022090800646,1
4,2022090800,818,40107,1,0,0,0,2022090800818,1
...,...,...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1,20220911132494,1
17422,2022092502,3510,42406,0,0,0,1,20220925023510,3
17423,2022091113,3642,43478,0,0,0,1,20220911133642,1
17424,2022091901,3578,42431,0,0,0,1,20220919013578,2


In [19]:
week1

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,2022-09-12 23:05:57.799999,NaN,football,left,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle
1407435,2022091200,3826,NaN,football,50,2022-09-12 23:05:57.900000,NaN,football,left,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN
1407436,2022091200,3826,NaN,football,51,2022-09-12 23:05:58.000000,NaN,football,left,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN
1407437,2022091200,3826,NaN,football,52,2022-09-12 23:05:58.099999,NaN,football,left,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN


In [20]:
week1['gamePlayId'] = week1[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
week1['gamePlayId'] = week1['gamePlayId'].astype('int64')

In [21]:
week1.drop(columns=['time', 'jerseyNumber', 'playDirection'], inplace=True)

In [22]:
week1 = pd.merge(week1, tackles, on=['gameId','playId','nflId','gamePlayId'], how='left')

In [23]:
week1 = week1.drop('week', axis=1)

In [24]:
week1 = pd.merge(week1, players, on=['nflId','displayName'], how='left')

In [25]:
week1.drop(['height', 'weight', 'birthDate', 'collegeName'], axis=1, inplace=True)

Before changing both handoffs and passes caught to a transfer event, a new column 'playType' will identify pass vs run plays based on these two event types. 

In [26]:
week1['playType'] = np.where(week1['event']=='handoff', 'run', 0)

In [27]:
week1['playType'] = np.where(week1['event']=='pass_outcome_caught', 'pass', week1['playType'])

In [28]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# To Expand Play Types To All Rows in Each gamePlayId Group

In [29]:
week1_playtype = week1[['gamePlayId', 'playType']]

In [30]:
week1_playtype

,gamePlayId,playType
0,202209080056,0
1,202209080056,0
2,202209080056,0
3,202209080056,0
4,202209080056,0
...,...,...
1407434,20220912003826,0
1407435,20220912003826,0
1407436,20220912003826,0
1407437,20220912003826,0


In [31]:
grouped_playtype = week1_playtype.groupby(['gamePlayId']).max().reset_index()

In [32]:
grouped_playtype

,gamePlayId,playType
0,202209080056,pass
1,202209080080,0
2,202209110057,0
3,202209110078,run
4,202209110185,pass
...,...,...
1468,20220912003628,pass
1469,20220912003685,run
1470,20220912003723,pass
1471,20220912003747,pass


In [33]:
grouped_playtype['playType'].value_counts()

pass    688
run     663
0       122
Name: playType, dtype: int64

In [34]:
grouped_playtype['playType'] = np.where(grouped_playtype['playType']=='0', 'direct', grouped_playtype['playType'])

In [35]:
grouped_playtype['playType'].value_counts()

pass      688
run       663
direct    122
Name: playType, dtype: int64

In [36]:
grouped_playtype

,gamePlayId,playType
0,202209080056,pass
1,202209080080,direct
2,202209110057,direct
3,202209110078,run
4,202209110185,pass
...,...,...
1468,20220912003628,pass
1469,20220912003685,run
1470,20220912003723,pass
1471,20220912003747,pass


In [37]:
week1 = pd.merge(week1, grouped_playtype, on=['gamePlayId'], how='left')

In [38]:
week1 = week1.drop('playType_x', axis=1)

In [39]:
week1.rename(columns={'playType_y':'playType'}, inplace=True)

In [40]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass


In [41]:
week1['playType'].value_counts()

run       837821
pass      444084
direct    125534
Name: playType, dtype: int64

# Developing a Ball Location Metric

In [42]:
pd.options.display.max_columns = None
plays.head(5)

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,gamePlayId,week
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,ATL,41,7:52,21,7,C,6.0,NaN,9,9,N,69,SHOTGUN,7.0,0.747284,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN,20221009083537,5
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,PIT,34,7:38,14,20,NaN,NaN,NaN,3,3,N,76,SHOTGUN,7.0,0.416454,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN,20220911033126,1
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,LV,30,8:57,10,3,C,11.0,NaN,15,15,N,40,I_FORM,6.0,0.267933,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN,20220911111148,1
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,DEN,37,13:12,19,16,NaN,NaN,NaN,7,7,N,47,SINGLEBACK,6.0,0.592704,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN,20221002122007,4
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,TEN,35,8:33,7,7,NaN,NaN,NaN,3,3,N,75,I_FORM,7.0,0.470508,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN,20220919001372,2


In [43]:
plays_merge = plays[['gameId','playId','gamePlayId','absoluteYardlineNumber','ballCarrierId','quarter','possessionTeam',
                     'defensiveTeam','playResult', 'down', 'yardsToGo', 'defendersInTheBox']]

In [44]:
plays_merge

,gameId,playId,gamePlayId,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox
0,2022100908,3537,20221009083537,69,48723,4,ATL,TB,9,1,10,7.0
1,2022091103,3126,20220911033126,76,52457,4,PIT,CIN,3,1,10,7.0
2,2022091111,1148,20220911111148,40,42547,2,LV,LAC,15,2,5,6.0
3,2022100212,2007,20221002122007,47,46461,3,DEN,LV,7,2,10,6.0
4,2022091900,1372,20220919001372,75,47857,2,BUF,TEN,3,1,10,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,2022100204123,49,43293,1,DAL,WAS,1,1,10,6.0
12482,2022091200,3467,20220912003467,40,46189,4,SEA,DEN,2,1,10,7.0
12483,2022101605,3371,20221016053371,69,44860,4,CIN,NO,6,1,10,6.0
12484,2022100207,2777,20221002072777,44,52449,3,IND,TEN,-2,1,10,6.0


In [45]:
week1 = pd.merge(week1, plays_merge, on=['gameId','playId','gamePlayId'], how='left')

In [46]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0


In [47]:
games_merge = games[['gameId', 'week', 'homeTeamAbbr', 'visitorTeamAbbr']]

In [48]:
week1 = pd.merge(week1, games_merge, on=['gameId'], how='left')

In [49]:
week1['nflId'] = week1['nflId'].fillna(0)

In [50]:
week1['nflId'] = week1['nflId'].astype(int)

In [51]:
week1['nflId'].isna().sum()

0

In [52]:
week1['xBallLocation'] = np.where(week1['ballCarrierId'] == week1['nflId'], week1['x'], 0)
week1['yBallLocation'] = np.where(week1['ballCarrierId'] == week1['nflId'], week1['y'], 0)

In [53]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0


To create the total yards on the play between the first frame and the last frame. 

In [54]:
week1['firstFrame'] = week1.groupby('gamePlayId')['frameId'].transform('min')

In [55]:
week1['lastFrame'] = week1.groupby('gamePlayId')['frameId'].transform('max')

In [56]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53


In [57]:
week1['xFirstFrame'] = np.where(week1['frameId'] == week1['firstFrame'], week1['xBallLocation'], 0)
week1['xLastFrame'] = np.where(week1['frameId'] == week1['lastFrame'], week1['xBallLocation'], 0)

In [58]:
week1['xFirstFrame'] = week1.groupby('gamePlayId')['xFirstFrame'].transform(max)
week1['xLastFrame'] = week1.groupby('gamePlayId')['xLastFrame'].transform(max)

In [59]:
week1['xFrameChange'] = (week1['xLastFrame'] - week1['xFirstFrame']).abs()

In [60]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99


Shows that there are 61,193 rows where the ball carrier ID matches the nfl ID. And below that it shows that there is a ball carrier ID on all the plays. So need to add the ball x and y to all the rows in the events dataset. 

In [61]:
len(week1.loc[week1.nflId == week1.ballCarrierId])

61193

In [62]:
ball_location = week1.loc[week1.nflId == week1.ballCarrierId]

In [63]:
ball_location

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
88,2022090800,56,42489,Stefon Diggs,1,BUF,80.60,38.29,6.23,0.90,0.62,140.05,189.38,NaN,202209080056,NaN,NaN,NaN,NaN,WR,72.0,1146.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09
89,2022090800,56,42489,Stefon Diggs,2,BUF,80.48,37.68,6.24,1.27,0.62,134.22,191.08,pass_arrived,202209080056,NaN,NaN,NaN,NaN,WR,72.0,1146.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09
90,2022090800,56,42489,Stefon Diggs,3,BUF,80.34,37.09,5.98,2.48,0.60,127.16,194.40,NaN,202209080056,NaN,NaN,NaN,NaN,WR,72.0,1146.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09
91,2022090800,56,42489,Stefon Diggs,4,BUF,80.19,36.53,5.67,3.37,0.58,120.72,196.97,NaN,202209080056,NaN,NaN,NaN,NaN,WR,72.0,1146.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09
92,2022090800,56,42489,Stefon Diggs,5,BUF,80.01,36.00,5.43,3.65,0.56,118.07,199.54,NaN,202209080056,NaN,NaN,NaN,NaN,WR,72.0,1146.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407275,2022091200,3826,53464,Javonte Williams,49,DEN,56.06,10.39,2.35,1.58,0.23,248.15,332.62,tackle,20220912003826,NaN,NaN,NaN,NaN,RB,70.0,1283.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99
1407276,2022091200,3826,53464,Javonte Williams,50,DEN,55.96,10.58,2.02,2.44,0.21,260.04,331.45,NaN,20220912003826,NaN,NaN,NaN,NaN,RB,70.0,1283.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99
1407277,2022091200,3826,53464,Javonte Williams,51,DEN,55.87,10.74,1.72,2.67,0.18,264.63,330.51,NaN,20220912003826,NaN,NaN,NaN,NaN,RB,70.0,1283.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99
1407278,2022091200,3826,53464,Javonte Williams,52,DEN,55.79,10.86,1.34,3.12,0.15,268.42,326.53,NaN,20220912003826,NaN,NaN,NaN,NaN,RB,70.0,1283.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99


In [64]:
ball_location = ball_location[['gamePlayId', 'frameId', 'xBallLocation', 'yBallLocation']]

In [65]:
ball_location

,gamePlayId,frameId,xBallLocation,yBallLocation
88,202209080056,1,80.60,38.29
89,202209080056,2,80.48,37.68
90,202209080056,3,80.34,37.09
91,202209080056,4,80.19,36.53
92,202209080056,5,80.01,36.00
...,...,...,...,...
1407275,20220912003826,49,56.06,10.39
1407276,20220912003826,50,55.96,10.58
1407277,20220912003826,51,55.87,10.74
1407278,20220912003826,52,55.79,10.86


In [66]:
week1 = pd.merge(week1, ball_location, on=['gamePlayId', 'frameId', 'xBallLocation', 'yBallLocation'], how='left')

In [67]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99


In [68]:
week1['xBallLocation'] = week1.groupby(['gamePlayId', 'frameId'])['xBallLocation'].transform(max)
week1['yBallLocation'] = week1.groupby(['gamePlayId', 'frameId'])['yBallLocation'].transform(max)

In [69]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99


In [70]:
week1['ballDistance'] = np.sqrt((week1['xBallLocation'] - week1['x']) ** 2 + (week1['yBallLocation'] - week1['y']) ** 2)

In [71]:
week1 = week1[week1.displayName != 'football']

In [72]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.75,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264
1407382,2022091200,3826,54618,Tariq Woolen,50,SEA,50.91,7.01,1.84,2.35,0.18,32.09,55.26,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,6.184448
1407383,2022091200,3826,54618,Tariq Woolen,51,SEA,51.06,7.13,1.85,1.98,0.18,32.09,49.15,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,6.014000
1407384,2022091200,3826,54618,Tariq Woolen,52,SEA,51.19,7.26,1.85,1.69,0.19,32.09,43.43,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,5.841233


In [73]:
week1['ballDistance'].max()

77.95014432315057

In [74]:
count = week1['ballDistance'].lt(1).sum()
print(count)

101601


In [75]:
count = week1['ballDistance'].lt(.05).sum()
print(count)

60832


In [76]:
count = week1['ballDistance'].lt(.0001).sum()
print(count)

60760


# Setting Up A Player Direction vs Ball Carrier Direction

In [77]:
week1['dirBallCarrier'] = np.where(week1['ballCarrierId'] == week1['nflId'], week1['dir'], 0)

In [78]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,0.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,0.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,0.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,0.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.75,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,0.0
1407382,2022091200,3826,54618,Tariq Woolen,50,SEA,50.91,7.01,1.84,2.35,0.18,32.09,55.26,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,6.184448,0.0
1407383,2022091200,3826,54618,Tariq Woolen,51,SEA,51.06,7.13,1.85,1.98,0.18,32.09,49.15,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,6.014000,0.0
1407384,2022091200,3826,54618,Tariq Woolen,52,SEA,51.19,7.26,1.85,1.69,0.19,32.09,43.43,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,5.841233,0.0


In [79]:
week1['dirBallCarrier'] = week1.groupby(['gamePlayId', 'frameId'])['dirBallCarrier'].transform(max)

In [80]:
week1['dirBallVariance'] = np.minimum(((week1['dir'] - week1['dirBallCarrier']).abs()), (360-(week1['dir'] - week1['dirBallCarrier']).abs()))

In [81]:
pd.options.display.max_columns = None
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,191.08,42.55
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,194.40,47.35
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,196.97,51.55
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,199.54,57.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.75,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45
1407382,2022091200,3826,54618,Tariq Woolen,50,SEA,50.91,7.01,1.84,2.35,0.18,32.09,55.26,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,6.184448,331.45,83.81
1407383,2022091200,3826,54618,Tariq Woolen,51,SEA,51.06,7.13,1.85,1.98,0.18,32.09,49.15,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,6.014000,330.51,78.64
1407384,2022091200,3826,54618,Tariq Woolen,52,SEA,51.19,7.26,1.85,1.69,0.19,32.09,43.43,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,5.841233,326.53,76.90


In [82]:
week1['momentum'] = (week1['s'] * week1['mass']).round()
week1['force'] = (week1['a'] * week1['mass']).round()

In [83]:
week1['momemtumVector'] = (week1['momentum'] * week1['dirBallVariance']).round()
week1['forceVector'] = (week1['force'] * week1['dirBallVariance']).round()

# Create a Snap Event at FrameId 1

In [84]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,191.08,42.55,3483.0,1272.0,148202.0,54124.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,194.40,47.35,3274.0,1022.0,155024.0,48392.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,196.97,51.55,3003.0,1856.0,154805.0,95677.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,199.54,57.59,2690.0,2586.0,154917.0,148928.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.75,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0
1407382,2022091200,3826,54618,Tariq Woolen,50,SEA,50.91,7.01,1.84,2.35,0.18,32.09,55.26,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,6.184448,331.45,83.81,2389.0,3051.0,200222.0,255704.0
1407383,2022091200,3826,54618,Tariq Woolen,51,SEA,51.06,7.13,1.85,1.98,0.18,32.09,49.15,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,6.014000,330.51,78.64,2402.0,2571.0,188893.0,202183.0
1407384,2022091200,3826,54618,Tariq Woolen,52,SEA,51.19,7.26,1.85,1.69,0.19,32.09,43.43,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,5.841233,326.53,76.90,2402.0,2194.0,184714.0,168719.0


In [85]:
week1 = week1[week1.event !='ball_snap']

In [86]:
week1['event'] = np.where(week1['frameId']==1, 'snap', week1['event'])

# Creating Yards from Different Events on Different Rows

In [87]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,191.08,42.55,3483.0,1272.0,148202.0,54124.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,194.40,47.35,3274.0,1022.0,155024.0,48392.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,196.97,51.55,3003.0,1856.0,154805.0,95677.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,199.54,57.59,2690.0,2586.0,154917.0,148928.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.75,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0
1407382,2022091200,3826,54618,Tariq Woolen,50,SEA,50.91,7.01,1.84,2.35,0.18,32.09,55.26,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,6.184448,331.45,83.81,2389.0,3051.0,200222.0,255704.0
1407383,2022091200,3826,54618,Tariq Woolen,51,SEA,51.06,7.13,1.85,1.98,0.18,32.09,49.15,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,6.014000,330.51,78.64,2402.0,2571.0,188893.0,202183.0
1407384,2022091200,3826,54618,Tariq Woolen,52,SEA,51.19,7.26,1.85,1.69,0.19,32.09,43.43,NaN,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,5.841233,326.53,76.90,2402.0,2194.0,184714.0,168719.0


In [88]:
week1['event'] = np.where(week1['event']=='pass_outcome_caught', 'transfer', week1['event'])

In [89]:
week1['event'] = np.where(week1['event']=='handoff', 'transfer', week1['event'])

In [90]:
week1_events = week1[week1['event'].notna()]

In [91]:
week1_events.rename(columns={'tackle':'tackler'}, inplace=True)

In [92]:
drop_events = ['pass_arrived', 'run', 'man_in_motion', 'play_action', 'shift', 'qb_slide', 'pass_forward', 'touchdown', 
               'lateral', 'line_set', 'snap_direct', 'autoevent_ballsnap', 'fumble_offense_recovered', 'run_pass_option', 
               'pass_shovel', 'fumble_defense_recovered', 'qb_sack', 'autoevent_passinterrupted', 'autoevent_passforward']

In [93]:
week1_events = week1_events[~week1_events.event.isin(drop_events)]

In [94]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,17.506050,178.63,101.41,7076.0,3025.0,717577.0,306765.0
1407338,2022091200,3826,54618,Tariq Woolen,6,SEA,51.22,16.13,3.22,4.63,0.34,335.44,296.50,transfer,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,19.794226,192.49,104.01,4181.0,6011.0,434866.0,625204.0
1407356,2022091200,3826,54618,Tariq Woolen,24,SEA,48.19,13.00,4.86,1.28,0.49,181.29,195.61,first_contact,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,17.903765,260.10,64.49,6310.0,1662.0,406932.0,107182.0


In [95]:
week1_events_merge = week1_events[['gamePlayId', 'event']]

In [96]:
week1_events_merge

,gamePlayId,event
0,202209080056,snap
5,202209080056,transfer
17,202209080056,tackle
22,202209080056,snap
27,202209080056,transfer
...,...,...
1407328,20220912003826,tackle
1407333,20220912003826,snap
1407338,20220912003826,transfer
1407356,20220912003826,first_contact


In [97]:
event_dummies = pd.get_dummies(week1_events_merge['event'])

In [98]:
event_dummies

,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,0,0,0,1,0,0
5,0,0,0,0,0,1
17,0,0,0,0,1,0
22,0,0,0,1,0,0
27,0,0,0,0,0,1
...,...,...,...,...,...,...
1407328,0,0,0,0,1,0
1407333,0,0,0,1,0,0
1407338,0,0,0,0,0,1
1407356,1,0,0,0,0,0


In [99]:
week1_events_merge = pd.concat([week1_events_merge, event_dummies], axis=1) 
week1_events_merge.drop(['event'], inplace=True, axis=1)
week1_events_merge.drop(['gamePlayId'], inplace=True, axis=1)

In [100]:
week1_events_merge

,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,0,0,0,1,0,0
5,0,0,0,0,0,1
17,0,0,0,0,1,0
22,0,0,0,1,0,0
27,0,0,0,0,0,1
...,...,...,...,...,...,...
1407328,0,0,0,0,1,0
1407333,0,0,0,1,0,0
1407338,0,0,0,0,0,1
1407356,1,0,0,0,0,0


In [101]:
week1_events_merge.apply(pd.Series.value_counts)

,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,93456,120384,116160,88572,95216,91102
1,27522,594,4818,32406,25762,29876


In [102]:
week1_events = pd.concat([week1_events, week1_events_merge], axis=1)

In [103]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0,0,0,0,0,1,0
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,17.506050,178.63,101.41,7076.0,3025.0,717577.0,306765.0,0,0,0,1,0,0
1407338,2022091200,3826,54618,Tariq Woolen,6,SEA,51.22,16.13,3.22,4.63,0.34,335.44,296.50,transfer,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,19.794226,192.49,104.01,4181.0,6011.0,434866.0,625204.0,0,0,0,0,0,1
1407356,2022091200,3826,54618,Tariq Woolen,24,SEA,48.19,13.00,4.86,1.28,0.49,181.29,195.61,first_contact,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,17.903765,260.10,64.49,6310.0,1662.0,406932.0,107182.0,1,0,0,0,0,0


In [104]:
week1_events['xSnap'] = np.where(week1_events['snap'] == 1, week1_events['x'], 0)
week1_events['ySnap'] = np.where(week1_events['snap'] == 1, week1_events['y'], 0)
week1_events['xContact'] = np.where(week1_events['first_contact'] == 1, week1_events['x'], 0)
week1_events['yContact'] = np.where(week1_events['first_contact'] == 1, week1_events['y'], 0)
week1_events['xTransfer'] = np.where(week1_events['transfer'] == 1, week1_events['x'], 0)
week1_events['yTransfer'] = np.where(week1_events['transfer'] == 1, week1_events['y'], 0)
week1_events['xTackle'] = np.where(week1_events['tackle'] == 1, week1_events['x'], 0)
week1_events['yTackle'] = np.where(week1_events['tackle'] == 1, week1_events['y'], 0)

In [105]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.37,27.27,0.00,0.0,0.00,0.00,0.00,0.00
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,0.00,0.00,0.00,0.0,88.80,26.70,0.00,0.00
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,0.00,0.00,0.00,0.0,0.00,0.00,88.23,27.09
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.25,27.53,0.00,0.0,0.00,0.00,0.00,0.00
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,0.00,0.00,0.00,0.0,78.11,28.74,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0,0,0,0,0,1,0,0.00,0.00,0.00,0.0,0.00,0.00,56.73,7.82
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,17.506050,178.63,101.41,7076.0,3025.0,717577.0,306765.0,0,0,0,1,0,0,53.28,15.53,0.00,0.0,0.00,0.00,0.00,0.00
1407338,2022091200,3826,54618,Tariq Woolen,6,SEA,51.22,16.13,3.22,4.63,0.34,335.44,296.50,transfer,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,19.794226,192.49,104.01,4181.0,6011.0,434866.0,625204.0,0,0,0,0,0,1,0.00,0.00,0.00,0.0,51.22,16.13,0.00,0.00
1407356,2022091200,3826,54618,Tariq Woolen,24,SEA,48.19,13.00,4.86,1.28,0.49,181.29,195.61,first_contact,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,17.903765,260.10,64.49,6310.0,1662.0,406932.0,107182.0,1,0,0,0,0,0,0.00,0.00,48.19,13.0,0.00,0.00,0.00,0.00


To mark each event location for each player for all snaps. 

In [106]:
event_locations = ['xSnap', 'ySnap', 'xContact', 'yContact', 'xTransfer', 'yTransfer', 'xTackle', 'yTackle']

In [107]:
for i in event_locations:
    week1_events[i] = week1_events.groupby(['gamePlayId', 'nflId'])[i].transform(max)

In [108]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0,0,0,0,0,1,0,71.48,27.14,66.21,14.23,71.56,25.09,56.73,7.82
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,17.506050,178.63,101.41,7076.0,3025.0,717577.0,306765.0,0,0,0,1,0,0,53.28,15.53,48.19,13.00,51.22,16.13,50.75,6.92
1407338,2022091200,3826,54618,Tariq Woolen,6,SEA,51.22,16.13,3.22,4.63,0.34,335.44,296.50,transfer,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,19.794226,192.49,104.01,4181.0,6011.0,434866.0,625204.0,0,0,0,0,0,1,53.28,15.53,48.19,13.00,51.22,16.13,50.75,6.92
1407356,2022091200,3826,54618,Tariq Woolen,24,SEA,48.19,13.00,4.86,1.28,0.49,181.29,195.61,first_contact,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,17.903765,260.10,64.49,6310.0,1662.0,406932.0,107182.0,1,0,0,0,0,0,53.28,15.53,48.19,13.00,51.22,16.13,50.75,6.92


In [109]:
week1_events['snapToTransfer'] = np.where((week1_events['xSnap'] > 0) & (week1_events['xTransfer'] > 0),
                                       (np.sqrt((week1_events['xTransfer'] - week1_events['xSnap']) ** 2 + (week1_events['yTransfer'] - week1_events['ySnap']) ** 2)), 0)

week1_events['transferToContact'] = np.where((week1_events['xTransfer'] > 0) & (week1_events['xContact'] > 0),
                                       (np.sqrt((week1_events['xContact'] - week1_events['xTransfer']) ** 2 + (week1_events['yContact'] - week1_events['yTransfer']) ** 2)), 0)

week1_events['transferToTackle'] = np.where((week1_events['xTransfer'] > 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xTransfer']) ** 2 + (week1_events['yTackle'] - week1_events['yTransfer']) ** 2)), 0)

week1_events['contactToTackle'] = np.where((week1_events['xContact'] > 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xContact']) ** 2 + (week1_events['yTackle'] - week1_events['yContact']) ** 2)), 0)

week1_events['snapToTackle'] = np.where((week1_events['xSnap'] > 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xSnap']) ** 2 + (week1_events['yTackle'] - week1_events['ySnap']) ** 2)), 0)

week1_events['snapToContact'] = np.where((week1_events['xSnap'] > 0) & (week1_events['xContact'] > 0),
                                       (np.sqrt((week1_events['xContact'] - week1_events['xSnap']) ** 2 + (week1_events['yContact'] - week1_events['ySnap']) ** 2)), 0)
                                     

In [110]:
week1_events['transferToTackle'] = np.where((week1_events['xTransfer'] == 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xSnap']) ** 2 + (week1_events['yTackle'] - week1_events['ySnap']) ** 2)),
                                           week1_events['transferToTackle'])

In [111]:
team_yards = week1_events.groupby('defensiveTeam').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
team_yards_sorted = team_yards.sort_values(by='defensiveTeam')

team_yards_sorted.head(10)

,defensiveTeam,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,ARI,2.191981,5.879474,6.397171,4.278120,3.029688,5.482662
1,ATL,1.936784,4.982726,6.394157,2.496772,2.731982,5.112354
2,BAL,2.103873,5.516296,6.403291,3.862042,2.664681,5.334974
3,BUF,1.789053,4.655698,6.416745,3.031935,2.900407,5.373865
4,CAR,2.350210,6.353629,7.723511,4.303552,2.875616,6.347417
5,CHI,1.936366,6.740923,8.032476,4.104399,3.120319,7.094809
6,CIN,1.844637,4.245766,6.397391,2.920890,1.887130,5.318955
7,CLE,2.017974,5.936776,8.346680,4.429775,3.547779,7.290520
8,DAL,1.866154,5.238986,6.580549,3.942403,2.510946,5.594450
9,DEN,1.835431,5.046597,6.106687,2.936355,2.847470,5.247138


In [112]:
tackler_yards = week1_events.groupby('tackler').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
tackler_yards_sorted = tackler_yards.sort_values(by='tackler')

tackler_yards_sorted

,tackler,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,0.0,2.083648,6.393825,8.322603,4.540181,3.755090,7.329454
1,1.0,2.256272,6.487223,7.568740,4.350821,2.849093,6.364186


In [113]:
ballDistance_yards = week1_events.groupby('tackler').agg(
    BallDistance=('ballDistance', 'mean'),

    
).reset_index()

# Sort teams by total tackles in descending order
ballDistance_yards_sorted = ballDistance_yards.sort_values(by='tackler')

ballDistance_yards_sorted

,tackler,BallDistance
0,0.0,6.218097
1,1.0,5.201252


In [114]:
ballDistanceAssist_yards = week1_events.groupby('assist').agg(
    BallDistance=('ballDistance', 'mean'),

    
).reset_index()

# Sort teams by total tackles in descending order
ballDistanceAssist_yards_sorted = ballDistanceAssist_yards.sort_values(by='assist')

ballDistanceAssist_yards_sorted

,assist,BallDistance
0,0.0,5.533681
1,1.0,5.840908


In [115]:
play_yards = week1_events.groupby('gamePlayId').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
play_yards_sorted = play_yards.sort_values(by='gamePlayId')

play_yards_sorted.head(15)

,gamePlayId,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,202209080056,1.437687,0.000000,4.032222,0.000000,0.000000,2.920850
1,202209080080,0.000000,5.531465,7.659848,0.000000,2.484230,7.659848
2,202209110057,0.000000,15.520742,17.618767,0.000000,2.554257,17.618767
3,202209110078,3.384356,7.867916,8.504963,6.869058,5.114808,9.083401
4,202209110185,1.926829,2.878276,5.889387,1.006387,3.865463,4.661372
5,202209110286,2.083275,7.112557,11.271590,5.483274,4.965766,9.791574
6,202209110358,1.756382,2.779680,5.015959,1.079273,2.461710,3.486728
7,202209110382,2.029245,0.000000,0.000000,0.000000,0.000000,0.000000
8,202209110458,2.022198,3.589867,5.936051,1.647606,2.581064,4.165385
9,202209110486,2.375027,5.402572,32.456726,3.713353,30.977813,32.497791


In [116]:
play_yards = week1_events.groupby('gamePlayId').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
play_yards_sorted = play_yards.sort_values(by='gamePlayId')

play_yards_sorted.head(15)

,gamePlayId,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,202209080056,1.437687,0.000000,4.032222,0.000000,0.000000,2.920850
1,202209080080,0.000000,5.531465,7.659848,0.000000,2.484230,7.659848
2,202209110057,0.000000,15.520742,17.618767,0.000000,2.554257,17.618767
3,202209110078,3.384356,7.867916,8.504963,6.869058,5.114808,9.083401
4,202209110185,1.926829,2.878276,5.889387,1.006387,3.865463,4.661372
5,202209110286,2.083275,7.112557,11.271590,5.483274,4.965766,9.791574
6,202209110358,1.756382,2.779680,5.015959,1.079273,2.461710,3.486728
7,202209110382,2.029245,0.000000,0.000000,0.000000,0.000000,0.000000
8,202209110458,2.022198,3.589867,5.936051,1.647606,2.581064,4.165385
9,202209110486,2.375027,5.402572,32.456726,3.713353,30.977813,32.497791


In [117]:
table = pd.pivot_table(week1_events, values='transferToTackle', index=['defensiveTeam'],
                       columns=['quarter'], aggfunc="mean", margins="True", margins_name='Overall')
table = table.assign(sortkey=table.index == 'Overall')\
                .sort_values(['sortkey','Overall'], ascending=[False, True])\
                .drop('sortkey', axis=1)
table

quarter,1,2,3,4,5,Overall
defensiveTeam,,,,,,
Overall,6.143766,5.294241,5.937998,5.481263,4.353428,5.662666
MIA,3.645721,6.469751,4.443261,4.974257,NaN,4.568462
TB,6.221620,3.491787,6.257969,3.187825,NaN,4.899319
LAC,4.798960,4.267454,7.408107,2.704261,NaN,4.923714
KC,8.224900,1.650809,5.631053,5.268195,NaN,4.942767
WAS,5.902473,4.901660,4.974351,3.865056,NaN,4.968900
IND,4.841168,5.566006,4.083473,8.221886,1.433384,4.998364
PIT,6.464715,4.493281,6.074627,4.851794,2.927400,5.107358
ATL,5.119366,4.026668,7.125260,3.268914,NaN,5.112354


In [118]:
table = pd.pivot_table(week1_events, values='transferToTackle', index=['defensiveTeam'],
                       columns=['playType'], aggfunc="mean", margins="True", margins_name='Overall')
table = table.assign(sortkey=table.index == 'Overall')\
                .sort_values(['sortkey','Overall'], ascending=[False, True])\
                .drop('sortkey', axis=1)
table

playType,direct,pass,run,Overall
defensiveTeam,,,,
Overall,6.701565,4.360339,6.825139,5.662666
MIA,NaN,3.210572,6.002853,4.568462
TB,9.772875,3.031291,7.157170,4.899319
LAC,NaN,4.234390,6.023699,4.923714
KC,6.369454,4.318372,5.498318,4.942767
WAS,2.819391,2.668354,8.056501,4.968900
IND,1.897912,3.802408,6.093317,4.998364
PIT,8.058763,4.092202,5.812589,5.107358
ATL,5.894999,3.318010,7.227795,5.112354


# To Track the Ball For Each Event for True Tackling Success

In [119]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0,0,0,0,0,1,0,71.48,27.14,66.21,14.23,71.56,25.09,56.73,7.82,2.051560,12.106283,22.763607,11.443710,24.306890,13.944210
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,17.506050,178.63,101.41,7076.0,3025.0,717577.0,306765.0,0,0,0,1,0,0,53.28,15.53,48.19,13.00,51.22,16.13,50.75,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101
1407338,2022091200,3826,54618,Tariq Woolen,6,SEA,51.22,16.13,3.22,4.63,0.34,335.44,296.50,transfer,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,19.794226,192.49,104.01,4181.0,6011.0,434866.0,625204.0,0,0,0,0,0,1,53.28,15.53,48.19,13.00,51.22,16.13,50.75,6.92,2.1456

In [120]:
week1_events['xBallSnap'] = np.where(week1_events['snap'] == 1, week1_events['xBallLocation'], 0)
week1_events['xBallTransfer'] = np.where(week1_events['transfer'] == 1, week1_events['xBallLocation'], 0)
week1_events['xBallContact'] = np.where(week1_events['first_contact'] == 1, week1_events['xBallLocation'], 0)
week1_events['xBallTackle'] = np.where(week1_events['tackle'] == 1, week1_events['xBallLocation'], 0)

In [121]:
pd.options.display.max_columns = None
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact,xBallSnap,xBallTransfer,xBallContact,xBallTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,0.00,0.00,0.00
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,0.00,79.85,0.00,0.00
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,0.00,0.00,0.00,79.33
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,0.00,0.00,0.00
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,0.00,79.85,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0,0,0,0,0,1,0,71.48,27.14,66.21,14.23,71.56,25.09,56.73,7.82,2.051560,12.106283,22.763607,11.443710,24.306890,13.944210,0.00,0.00,0.00,56.06
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,17.506050,178.63,101.41,7076.0,3025.0,717577.0,306765.0,0,0,0,1,0,0,53.28,15.53,48.19,13.00,51.22,16.13,50.75,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101,70.71,0.00,0.00,0.00
1407338,2022091200,3826,54618,Tariq Woolen,6,SEA,51.22,16.13,3.22,4.63,0.34,335.44,296.50,transfer,20220912003826,NaN,NaN,NaN,NaN,CB,76

In [122]:
events_ball = ['xBallSnap', 'xBallTransfer', 'xBallContact', 'xBallTackle']

In [123]:
for i in events_ball:
    week1_events[i] = week1_events.groupby(['gamePlayId'])[i].transform(max)

In [124]:
week1_events = week1_events[week1_events['xBallTackle']!=0]

In [125]:
week1_events['xBallTransfer'] = np.where(week1_events['xBallTransfer']== 0, week1_events['xBallSnap'], week1_events['xBallTransfer'])
week1_events['xBallContact'] = np.where(week1_events['xBallContact']== 0, week1_events['xBallTackle'], week1_events['xBallContact'])

In [126]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact,xBallSnap,xBallTransfer,xBallContact,xBallTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0,0,0,0,0,1,0,71.48,27.14,66.21,14.23,71.56,25.09,56.73,7.82,2.051560,12.106283,22.763607,11.443710,24.306890,13.944210,70.71,70.57,64.41,56.06
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,17.506050,178.63,101.41,7076.0,3025.0,717577.0,306765.0,0,0,0,1,0,0,53.28,15.53,48.19,13.00,51.22,16.13,50.75,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101,70.71,70.57,64.41,56.06
1407338,2022091200,3826,54618,Tariq Woolen,6,SEA,51.22,16.13,3.22,4.63,0.34,335.44,296.50,transfer,20220912003826,

In [127]:
week1_events['ballSnapToTransfer'] = (week1_events['xBallTransfer'] - week1_events['xBallSnap']).abs()
week1_events['ballSnapToContact'] = (week1_events['xBallContact'] - week1_events['xBallSnap']).abs()
week1_events['ballTransferToContact'] = (week1_events['xBallContact'] - week1_events['xBallTransfer']).abs()
week1_events['ballContactToTackle'] = (week1_events['xBallTackle'] - week1_events['xBallContact']).abs()
week1_events['ballSnapToTackle'] = (week1_events['xBallTackle'] - week1_events['xBallSnap']).abs()
week1_events['ballTransferToTackle'] = (week1_events['xBallTackle'] - week1_events['xBallTransfer']).abs()
week1_events['ballTackleToEnd'] = (week1_events['xLastFrame'] - week1_events['xBallTackle']).abs()

In [128]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,down,yardsToGo,defendersInTheBox,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact,xBallSnap,xBallTransfer,xBallContact,xBallTackle,ballSnapToTransfer,ballSnapToContact,ballTransferToContact,ballContactToTackle,ballSnapToTackle,ballTransferToTackle,ballTackleToEnd
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.23,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.37,27.27,0.00,0.00,88.80,26.70,88.23,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
22,2022090800,56,38577,Bobby Wagner,1,LA,78.25,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
27,2022090800,56,38577,Bobby Wagner,6,LA,78.11,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,10,6.0,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.25,27.53,0.00,0.00,78.11,28.74,78.62,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.73,7.82,0.98,2.21,0.11,17.90,340.05,tackle,20220912003826,NaN,NaN,NaN,NaN,OLB,76.0,1678.33,pass,65,53464,4,DEN,SEA,9,3,14,3.0,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899,332.62,7.43,1645.0,3709.0,12222.0,27558.0,0,0,0,0,1,0,71.48,27.14,66.21,14.23,71.56,25.09,56.73,7.82,2.051560,12.106283,22.763607,11.443710,24.306890,13.944210,70.71,70.57,64.41,56.06,0.14,6.30,6.16,8.35,14.65,14.51,0.34
1407333,2022091200,3826,54618,Tariq Woolen,1,SEA,53.28,15.53,5.45,2.33,0.55,315.69,280.04,snap,20220912003826,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,

In [129]:
play_yards = week1_events.groupby('gamePlayId').agg(
    PlayResult=('playResult', max),
    FrameChange=('xFrameChange', max),
    BallSnapToTransfer=('ballSnapToTransfer', max),
    BallSnapToContact=('ballSnapToContact', max),
    BallSnapToTackle=('ballSnapToTackle', max),
    BallTransferToContact=('ballTransferToContact', max),
    BallContactToTackle=('ballContactToTackle', max),
    BallTransferToTackle=('ballTransferToTackle', max),
    BallTackletoEnd=('ballTackleToEnd', max)
    
).reset_index()

# Sort teams by total tackles in descending order
play_yards_sorted = play_yards.sort_values(by='gamePlayId')

play_yards_sorted.head(15)

,gamePlayId,PlayResult,FrameChange,BallSnapToTransfer,BallSnapToContact,BallSnapToTackle,BallTransferToContact,BallContactToTackle,BallTransferToTackle,BallTackletoEnd
0,202209080056,6,1.09,0.75,1.27,1.27,0.52,0.00,0.52,0.18
1,202209080080,7,11.47,0.00,8.27,10.70,8.27,2.43,10.70,0.77
2,202209110057,11,18.24,0.00,13.21,16.59,13.21,3.38,16.59,1.65
3,202209110078,6,13.94,0.78,10.89,13.61,10.11,2.72,12.83,0.33
4,202209110185,-5,3.08,0.14,0.30,3.11,0.44,2.81,3.25,0.03
5,202209110286,3,7.91,0.17,5.30,8.20,5.13,2.90,8.03,0.29
6,202209110358,3,4.94,1.34,2.30,4.51,0.96,2.21,3.17,0.43
7,202209110458,-1,6.08,1.59,3.15,5.29,1.56,2.14,3.70,0.79
8,202209110486,50,58.80,2.39,10.47,56.37,8.08,45.90,53.98,2.43
9,202209110556,4,9.02,0.12,4.61,8.76,4.49,4.15,8.64,0.26


In [130]:
team_yards = week1_events.groupby('defensiveTeam').agg(
    TransferToTackleYards=('transferToTackle', 'mean'),
    BallTransferToTackle=('ballTransferToTackle', 'mean'),
    ForceVector=('forceVector', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
team_yards_sorted = team_yards.sort_values(by='defensiveTeam')

team_yards_sorted

,defensiveTeam,TransferToTackleYards,BallTransferToTackle,ForceVector
0,ARI,7.358309,8.237105,170621.181220
1,ATL,5.978155,6.595887,150321.428152
2,BAL,6.685183,5.571296,192062.682099
3,BUF,6.121223,5.851126,206531.547562
4,CAR,7.463446,7.123022,152357.507992
5,CHI,8.758419,8.647586,139566.692790
6,CIN,6.419428,6.417759,164223.936851
7,CLE,8.210682,8.987087,165887.688858
8,DAL,6.801096,7.158431,155951.767380
9,DEN,6.951336,7.205299,183247.054390
